目前为止，梯度学习的方法中有两个极端： 一次使用所有的数据计算梯度和更新参数；一次计算一次梯度。

## 1 向量化和缓存

决定使用小批量的主要原因是计算效率。当考虑并行化到多个GPU和多个服务器时很好解释。我们需要向每一个GPU至少发送一个图像，假设每个服务器8个GPU一共16个服务器，那么我们的最小批次已经达到了128。

在单个GPU甚至CPU来看，情况更加微妙。设备的内存类型千奇百怪，用于计算的单元也不尽相同，而且服务器之间的带宽限制也是不同的。例如，CPU的寄存器数量比较少，之后是L1,L2高速缓存，某些情况下还有L3（内核间的共享缓存）。处理器能够处理比主存储器接口提供的更多的任务：
+ 具有16个内核和AVX-512矢量化功能的2GHz CPU最多可以处理  $2⋅10^9⋅16⋅32=10^{12}$ 字节数每秒。GPU的功能很容易超过这个100倍。但是，中端服务器处理器的带宽可能不会超过100 GB / s，即不到保持处理器所需带宽的十分之一。
+ 随机开始访问的时候开销很大，而顺序一直访问则相对开销较小。

减轻这个约束的方法是使用CPU缓存的层次结构。层次结构够快能够应付处理器的数据需求。这是深度学习中批处理的底层驱动力。简单举例，矩阵相乘 $\mathbf{A} = \mathbf{B}\mathbf{C}$。我们有许多方法计算获得$\mathbf{A}$：

1.可以通过 $\mathbf{A}_{ij} = \mathbf{B}_{i,:} \mathbf{C}_{:,j}^\top$, 即：通过点积的方式逐个元素计算。
1. 可以通过 $\mathbf{A}_{:,j} = \mathbf{B} \mathbf{C}_{:,j}^\top$, 即,可以一次计算一列，同样也可以一次计算一排。
1. 可以简单的通过 $\mathbf{A} = \mathbf{B} \mathbf{C}$计算
1. 我们可以讲 $\mathbf{B}$ 和 $\mathbf{C}$ 拆分成较小的矩阵然后一次一块的计算 $\mathbf{A}$ 

按照第一个方法，我们每次计算元素的时候，每一次我们计算$\mathbf{A}_{ij}$时，都需要将一行和一列向量复制到CPU中。由于矩阵元素是按顺序排列的，我们从内存中两个向量时，需要访问他们不行相交的位置，造成读取上的浪费。 第二种方式更加合理，我们可以在遍历 $B$进行计算时保留列向量$\mathbf{C}_{:,j}$在CPU的缓存中。这样可以讲内存带宽需求减半因此更快。当然, 第三个方法最好，但是,大多数矩阵可能不适合进行全部缓存 。然而,方法4可以作为3的替代方法:我们可以将块矩阵进行缓存然后计算。

除了计算效率外，python语言和深度学习框架本身也会带来很大的开销。在每次执行命令时，Python解释器都要讲命令发送到MXNet引擎，引擎将其插入计算图中并在调度期间进行处理。这样的开销也是很大的，因此强烈建议使用向量和矩阵。

In [1]:
from d2l import mxnet as d2l
from mxnet import autograd, gluon, init, np, npx
from mxnet.gluon import nn
import plotly.express as px
import pandas as pd
npx.set_np()

timer = d2l.Timer()

A = np.zeros((256, 256))
B = np.random.normal(0, 1, (256, 256))
C = np.random.normal(0, 1, (256, 256))

通过第一种方法，按元素分配简单地遍历所有行和列。

In [2]:
timer.start()
for i in range(256):
    for j in range(256):
        A[i, j] = np.dot(B[i, :], C[:, j])
A.wait_to_read()
timer.stop()

39.027339696884155

使用第二种方法，按列进行分配，然后计算

In [3]:
timer.start()
for j in range(256):
    A[:, j] = np.dot(B, C[:, j])
A.wait_to_read()
timer.stop()

0.10952019691467285

使用第三个方法，将块缓存，然后计算。

In [4]:
timer.start()
A = np.dot(B, C)
A.wait_to_read()
timer.stop()

0.0006384849548339844

## 2.迷你批次

在之前的工作中我们认为使用 *minibatch* 来更新数据是理所当然的。 这里进行简单的说明，上面的例子已经说明，想要单独观测需要执行很多单个矩阵乘法,这样做开销是很大的。这个同时适用于评估网络，也适用于计算梯度。就是说, 我们执行$\mathbf{w} \leftarrow \mathbf{w} - \eta_t \mathbf{g}_t$ 在：

$$\mathbf{g}_t = \partial_{\mathbf{w}} f(\mathbf{x}_{t}, \mathbf{w})$$

我们可以提高计算效率通过将其应用于少量的观察。就是说, 我们更新梯度 $\mathbf{g}_t$通过一次观测一小批次。

$$\mathbf{g}_t = \partial_{\mathbf{w}} \frac{1}{|\mathcal{B}_t|} \sum_{i \in \mathcal{B}_t} f(\mathbf{x}_{i}, \mathbf{w})$$

决定 $\mathbf{g}_t$的统计: 随着$\mathbf{x}_t$ 和 minibatch $\mathcal{B}_t$从训练集中随机抽取, 梯度的期望值保存不变。 然而差异显著减小。由于小批量梯度是 $b := |\mathcal{B}_t|$ 独立的平均梯度, 它的标准偏差减小了 $b^{-\frac{1}{2}}$。

当然这也标明minibatch$\mathcal{B}_t$越大越好。但是，这也与计算成本成线性增加相比，这点偏差是值得的。在实践中，我们选择的迷你批处理足够大，可以提供良好的计算效率，同时仍适合GPU的内存。看下例子：但是这次分解为64列的“ minibatches”

In [5]:
timer.start()
for j in range(0, 256, 64):
    A[:, j:j+64] = np.dot(B, C[:, j:j+64])
timer.stop()

for i,  gigaflop in enumerate( [2/i for i in timer.times]):
    print(f'方法{i+1} :  {gigaflop:.3f}')

方法1 :  0.051
方法2 :  18.261
方法3 :  3132.415
方法4 :  741.567


## 3. 读取数据集

让我们看一下如何从数据有效地生成小批量。使用的是由NASA开发的数据集来测试：不同飞机的机翼噪声，来比较这些优化算法。为了方便，这里使用1500作为样本。

In [6]:
d2l.DATA_HUB['airfoil'] = (d2l.DATA_URL + 'airfoil_self_noise.dat', '76e5be1548fd8222e5074cf0faae75edff8cf93f')

def get_data(batch_size=10, n=1500):
    data = np.genfromtxt(d2l.download('airfoil'),  dtype=np.float32, delimiter='\t')
    data = (data - data.mean(axis=0)) / data.std(axis=0)
    data_iter = d2l.load_array((data[:n, :-1], data[:n, -1]), batch_size, is_train=True)
    return data_iter, data.shape[1]-1

## 4. 开始实现

创建sgd方法。在训练函数中对每个小批量例子的损失做了平均，因此优化算法中的梯度不再需要处理批量大小。

In [7]:
def sgd(params, states, hyperparams):
    for p in params:
        p[:] -= hyperparams['lr'] * p.grad

实现通用的训练函数：初始化线性回归模型，并可用于通过minibatch SGD和随后引入的其他算法来训练模型。

In [8]:
def train(trainer_fn, states, hyperparams, data_iter, feature_dim, num_epochs=2):
    # 初始化
    w = np.random.normal(scale=0.01, size=(feature_dim, 1))
    b = np.zeros(1)
    w.attach_grad()
    b.attach_grad()
    net, loss = lambda X: d2l.linreg(X, w, b), d2l.squared_loss
    # 训练
    n, timer = 0, d2l.Timer()
    epoch_lst, loss_lst  = [], []
    for _ in range(num_epochs):
        for X, y in data_iter:
            with autograd.record():
                l = loss(net(X), y).mean()
            l.backward()
            trainer_fn([w, b], states, hyperparams)
            n += X.shape[0]
            if n % 200 == 0:
                timer.stop()
                epoch_lst.append(n/X.shape[0]/len(data_iter))
                loss_lst.append(d2l.evaluate_loss(net, data_iter, loss))
                timer.start()
    print(f'loss: {loss_lst[-1]:.3f}, {timer.avg():.3f} sec/epoch')
    fig = px.line(pd.DataFrame([loss_lst], columns=epoch_lst, index=['loss']).T, width=600, height=380)
    fig.update_layout(xaxis_range=[0,num_epochs], yaxis_range=[0.22, 0.35], xaxis_title='epoch', yaxis_title='loss')
    fig.show()
    
    return timer.cumsum(), loss_lst

让我们看看批量梯度下降的优化过程。这可以通过将最小批量大小设置为1500（即示例总数）来实现。结果，模型参数每个时期仅更新一次。没有什么进展。实际上，经过6个步骤，进度就会停止。

In [9]:
def train_sgd(lr, batch_size, num_epochs=2):
    data_iter, feature_dim = get_data_ch11(batch_size)
    return train( sgd, None, {'lr': lr}, data_iter, feature_dim, num_epochs)

gd_res = train_sgd(1, 1500, 10)

NameError: name 'get_data_ch11' is not defined

当批量大小等于1时，我们使用SGD进行优化。为了简化实施，我们选择了一个恒定的（尽管很小）学习率。在SGD中，只要处理示例，模型参数就会更新。在我们的情况下，每个时期总计1500次更新。尽管这两个过程在一个时期内处理了1500个示例，但在我们的实验中，SGD比梯度下降消耗的时间更多。这是因为SGD更频繁地更新参数，并且一次处理一个观测值效率较低。

In [ ]:
sgd_res = train_sgd(0.005, 1)

最后，当批量大小等于100时，我们使用minibatch SGD进行优化。每个时期所需的时间短于SGD所需的时间和批次梯度下降所需的时间。

In [ ]:
mini1_res = train_sgd(.4, 100)

将批次大小减小到10，由于每个批次的工作负载执行效率较低，因此每个时期的时间增加了。

In [ ]:
mini2_res = train_sgd(.05, 10)

我们比较了预览四个实验的时间与损失。可以看出，SGD的收敛速度比GD快，但与GD相比，它花费了更多的时间达到相同的损失，因为逐个示例计算梯度效率不高。Minibatch SGD能够权衡收敛速度和计算效率。小批量10比SGD更有效；在运行时间方面，小批量100甚至胜过GD。

In [ ]:
import plotly.graph_objs as go
fig = go.Figure()
for (x, y), name in zip([gd_res, sgd_res, mini1_res, mini2_res], ['gd', 'sgd', 'batch size=100', 'batch size=10']):
    fig.add_trace(go.Scatter(x=x, y=y, name=name))
fig.update_layout(xaxis_type='log', width=600, height=380, xaxis_title='time(sec)', yaxis_title='loss')
fig.show()

## 5. 简化

在Gluon中，我们可以使用Trainer该类来调用优化算法。这用于实现通用训练功能。

In [ ]:
def train_concise(tr_name, hyperparams, data_iter, num_epochs=2):
    # 初始化
    net = nn.Sequential()
    net.add(nn.Dense(1))
    net.initialize(init.Normal(sigma=0.01))
    trainer = gluon.Trainer(net.collect_params(), tr_name, hyperparams)
    loss = gluon.loss.L2Loss()
    n, timer = 0, d2l.Timer()
    epoch_lst, loss_lst  = [], []
    for _ in range(num_epochs):
        for X, y in data_iter:
            with autograd.record():
                l = loss(net(X), y)
            l.backward()
            trainer.step(X.shape[0])
            n += X.shape[0]
            if n % 200 == 0:
                timer.stop()
                epoch_lst.append(n/X.shape[0]/len(data_iter))
                loss_lst.append(d2l.evaluate_loss(net, data_iter, loss))
                timer.start()
    print(f'loss: {loss_lst[-1]:.3f}, {timer.avg():.3f} sec/epoch')
    fig = px.line(pd.DataFrame([loss_lst], columns=epoch_lst, index=['loss']).T, width=600, height=380)
    fig.update_layout(xaxis_range=[0,num_epochs], yaxis_range=[0.22, 0.35], xaxis_title='epoch', yaxis_title='loss')
    fig.show()
    

使用Gluon重复上一个实验将显示相同的行为

In [ ]:
data_iter, _ = get_data(10)
train_concise('sgd', {'learning_rate': 0.05}, data_iter)